<a href="https://colab.research.google.com/github/YMGYM/TSE_Learning/blob/master/air_pollution_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load Data

First, I used the data about air_polution data from 
Ministry of Environment

In [ ]:
import pandas as pd
import json
import tensorflow as tf
import tensorflow.keras as K
import numpy as np

I got the tip from how to read the file with utf-8 encoding from [here](https://teddylee777.github.io/pandas/%EA%B3%B5%EA%B3%B5%EB%8D%B0%EC%9D%B4%ED%84%B0-%ED%95%9C%EA%B8%80%EA%B9%A8%EC%A7%90%ED%98%84%EC%83%81-%ED%95%B4%EA%B2%B0%EB%B0%A9%EB%B2%95)

In [ ]:
def read_data():
  data = pd.read_csv("/content/drive/My Drive/Datasets/air_2019.csv", encoding='utf-8')
  return data

In [ ]:
data = read_data()

In [ ]:
data

,측정일자,권역코드,권역명,측정소코드,측정소명,미세먼지(㎍/㎥),오존(ppm),이산화질소농도(ppm),일산화탄소농도(ppm),아황산가스농도(ppm),초미세먼지(㎍/㎥)
0,20191231,100,도심권,111121,중구,26,18,0.023,0.012,0.4,0.002
1,20191231,100,도심권,111123,종로구,27,19,0.021,0.015,0.4,0.003
2,20191231,100,도심권,111131,용산구,14,15,0.020,0.015,0.4,0.003
3,20191231,101,서북권,111181,은평구,24,13,0.024,0.010,0.4,0.003
4,20191231,101,서북권,111191,서대문구,23,9,0.023,0.013,0.5,0.002
...,...,...,...,...,...,...,...,...,...,...,...
8891,20190101,103,서남권,111301,양천구,46,29,0.014,0.033,0.6,0.002
8892,20190101,104,동남권,111261,강남구,37,31,0.012,0.028,0.6,0.005
8893,20190101,104,동남권,111262,서초구,37,20,0.015,0.032,0.4,0.004
8894,20190101,104,동남권,111273,송파구,44,28,0.011,0.042,0.8,0.004


Filtering data

In [ ]:
location = data['측정소명'] == "중구"

dust_location = data[location]
new_list = pd.DataFrame(dust_location['미세먼지(㎍/㎥)'])
new_list

,미세먼지(㎍/㎥)
0,26
25,36
50,46
75,43
100,34
...,...
8771,63
8796,55
8821,35
8846,31


In [ ]:
def read_dust_data():
  data = pd.read_excel("/content/drive/My Drive/Datasets/APIS/APIS_data.xlsx", encoding='utf-8')
  return data

In [ ]:
data = read_dust_data()

In [ ]:
data

,1.0ug/m3,2.5ug/m3
0,21,29
1,21,30
2,21,30
3,22,32
4,22,32
...,...,...
3327,25,37
3328,25,36
3329,26,37
3330,26,38


# 공공데이터 받기

In [62]:
import urllib
from urllib.parse import urlencode, quote_plus, unquote
from urllib.request import urlopen
import requests
import json


DECODE_KEY = unquote('cELHrfnETp70pjPK90A%2F%2B2h4u9%2FNYVQ6o4e%2B1icLTH6TodaaaxmUvLfsCsRVvH4XvI6NqZSl4jC3grykXKEVTg%3D%3D')


In [65]:
def api_Living_Reading(key, feature, time):
  #-*- coding:utf-8 -*-
  url = f"http://apis.data.go.kr/1360000/LivingWthrIdxService/{feature}"
  
  queryParams = '?' + urlencode({ quote_plus('ServiceKey') : key, quote_plus('pageNo') : '1', quote_plus('dataType') : 'JSON', quote_plus('areaNo') : '1100000000', quote_plus('time') : time,quote_plus('requestCode') : 'A20'})
  request = requests.get(url + queryParams)

  json_data = json.loads(request.text)
  return json_data["response"]['body']["items"]["item"][0]

In [66]:
items = api_Living_Reading(DECODE_KEY, 'getHeatFeelingIdx', '2020072706')

In [67]:
items

{'header': {'resultCode': '99', 'resultMsg': '검색결과가 없습니다. [1100000000]'}}

In [ ]:
heatfeeling = items['h3'] # 더위체감지수
heatfeeling

'23'

In [ ]:
items = api_Living_Reading(DECODE_KEY, 'getDiscomfortIdx', '2020072706')

In [ ]:
discomfort = items['h3'] # 불쾌지수
discomfort

'73'

In [ ]:
items = api_Living_Reading(DECODE_KEY, 'getUVIdx', '2020072723')

In [ ]:
uv = items['today'] # 자외선지수
uv

''

In [ ]:
items = api_Living_Reading(DECODE_KEY, 'getSenTaIdx', '2020072723')

In [ ]:
heat = items['h3'] # 체감온도
heat

'23'

## Time Module

In [1]:
import time

now = time.localtime()
now

time.struct_time(tm_year=2020, tm_mon=7, tm_mday=31, tm_hour=5, tm_min=3, tm_sec=25, tm_wday=4, tm_yday=213, tm_isdst=0)

In [2]:
DATE = f"{now.tm_year}{now.tm_mon:02d}{now.tm_mday:02d}{now.tm_hour:02d}"

In [3]:
DATE

'2020073105'

In [4]:
items = api_Living_Reading(DECODE_KEY, 'getDiscomfortIdx', DATE)

NameError: ignored

In [ ]:
items

In [ ]:
items = api_Living_Reading(DECODE_KEY, 'getAirDiffusionIdx', DATE)

In [ ]:
items['h3'] # 대기확산지수

## If time is same...

In [ ]:
lasttime = DATE

In [ ]:
if lasttime != DATE:
  lasttime = DATE

## overall function

In [5]:
import urllib
from urllib.parse import urlencode, quote_plus, unquote
from urllib.request import urlopen
import requests
import json
import time

In [43]:
now = time.localtime()

In [44]:
DECODE_KEY = unquote('cELHrfnETp70pjPK90A%2F%2B2h4u9%2FNYVQ6o4e%2B1icLTH6TodaaaxmUvLfsCsRVvH4XvI6NqZSl4jC3grykXKEVTg%3D%3D')
DATE = f"{now.tm_year}{now.tm_mon:02d}{now.tm_mday:02d}{now.tm_hour:02d}"
QUERY = ['getHeatFeelingIdx','getDiscomfortIdx','getUVIdx','getSenTaIdx','getAirDiffusionIdx']
lasttime = ""

In [45]:
def api_Living_Reading(key, feature, time, areano='1100000000'):
  #-*- coding:utf-8 -*-
  url = f"http://apis.data.go.kr/1360000/LivingWthrIdxService/{feature}"
  
  queryParams = '?' + urlencode({ quote_plus('ServiceKey') : key, quote_plus('pageNo') : '1', quote_plus('dataType') : 'JSON', quote_plus('areaNo') : areano, quote_plus('time') : time, quote_plus('requestCode') : 'A20'})
  request = requests.get(url + queryParams)
  json_data = json.loads(request.text)

  if feature == 'getUVIdx':
    return json_data["response"]['body']["items"]["item"][0]['today']
  else:
    return json_data["response"]['body']["items"]["item"][0]['h3']

In [46]:
def get_living_weather_data(key, query, time, lasttime,result, areano='1100000000'):
  if lasttime == DATE:
    return result, lasttime
  else:
    lasttime = DATE
    print("Receiving now data")
    for feature in QUERY:
      result[feature] = api_Living_Reading(DECODE_KEY, feature, DATE)
    return result, lasttime

In [47]:
result, lasttime = get_living_weather_data(DECODE_KEY,QUERY,DATE,lasttime,result)

Receiving now data


In [48]:
result

{'CO': '0.4',
 'NO2': '0.016',
 'O3': '0.041',
 'PM10': '26',
 'PM25': '18',
 'SO2': '0.002',
 'getAirDiffusionIdx': '100',
 'getDiscomfortIdx': '74',
 'getHeatFeelingIdx': '24',
 'getSenTaIdx': '24',
 'getUVIdx': ''}

# Public Air Dust Data Read

In [12]:
import urllib
from urllib.parse import urlencode, quote_plus, unquote
from urllib.request import urlopen
import requests
import json
import time

In [51]:

url = f"http://openapi.airkorea.or.kr/openapi/services/rest/ArpltnInforInqireSvc/getCtprvnMesureLIst"

DECODE_KEY = unquote('cELHrfnETp70pjPK90A%2F%2B2h4u9%2FNYVQ6o4e%2B1icLTH6TodaaaxmUvLfsCsRVvH4XvI6NqZSl4jC3grykXKEVTg%3D%3D') 

#-*- coding:utf-8 -*-
queryParams = '?' + urlencode({ quote_plus('ServiceKey') : DECODE_KEY, quote_plus('itemCode') : 'PM10', quote_plus('dataGubun') : 'HOUR', quote_plus('_returnType') : 'json' })
request = requests.get(url + queryParams)

request



<Response [200]>

In [52]:
json_data = json.loads(request.text)

In [53]:
json_data['list'][0]

{'_returnType': 'json',
 'busan': '18',
 'chungbuk': '17',
 'chungnam': '11',
 'daegu': '18',
 'daejeon': '7',
 'dataGubun': '1',
 'dataTerm': '',
 'dataTime': '2020-07-31 14:00',
 'gangwon': '17',
 'gwangju': '14',
 'gyeongbuk': '14',
 'gyeonggi': '29',
 'gyeongnam': '17',
 'incheon': '36',
 'itemCode': '10007',
 'jeju': '15',
 'jeonbuk': '8',
 'jeonnam': '15',
 'numOfRows': '10',
 'pageNo': '1',
 'resultCode': '',
 'resultMsg': '',
 'searchCondition': '',
 'sejong': '6',
 'seoul': '28',
 'serviceKey': '',
 'totalCount': '',
 'ulsan': '20'}

## Automatically collect all data

In [26]:
ITEM_CODE = ['SO2', 'CO', 'O3', 'NO2', 'PM10', 'PM25']
DECODE_KEY = unquote('cELHrfnETp70pjPK90A%2F%2B2h4u9%2FNYVQ6o4e%2B1icLTH6TodaaaxmUvLfsCsRVvH4XvI6NqZSl4jC3grykXKEVTg%3D%3D') 

한국환경공단 에어코리아 openAPI를 사용했습니다
https://data.go.kr/tcs/dss/selectApiDataDetailView.do?publicDataPk=15000581

In [36]:
def api_air_pullution(key, item_code):
  url = f"http://openapi.airkorea.or.kr/openapi/services/rest/ArpltnInforInqireSvc/getCtprvnMesureLIst"


  #-*- coding:utf-8 -*-
  queryParams = '?' + urlencode({ quote_plus('ServiceKey') : key, quote_plus('itemCode') : item_code, quote_plus('dataGubun') : 'HOUR', quote_plus('_returnType') : 'json' })
  request = requests.get(url + queryParams)

  json_data = json.loads(request.text)
  return json_data['list'][0]


In [40]:
def get_air_pollution_data(key, item_code, station="seoul"):
  result = {}
  for item in item_code:
    value = api_air_pullution(key, item)
    result[item] = value[station]

  return result  

In [41]:
result = get_air_pollution_data(DECODE_KEY, ITEM_CODE, station='seoul')

In [42]:
result

{'CO': '0.4',
 'NO2': '0.016',
 'O3': '0.041',
 'PM10': '26',
 'PM25': '18',
 'SO2': '0.002'}

# Function Collection

In [1]:
import urllib
from urllib.parse import urlencode, quote_plus, unquote
from urllib.request import urlopen
import requests
import json
import time

def api_Living_Reading(key, feature, time, areano='1100000000'):\
  # I used api from Korea Meteorological Administration
  # https://data.go.kr/tcs/dss/selectApiDataDetailView.do?publicDataPk=15043583

  #-*- coding:utf-8 -*-
  url = f"http://apis.data.go.kr/1360000/LivingWthrIdxService/{feature}"
  
  queryParams = '?' + urlencode({ quote_plus('ServiceKey') : key, quote_plus('pageNo') : '1', quote_plus('dataType') : 'JSON', quote_plus('areaNo') : areano, quote_plus('time') : time, quote_plus('requestCode') : 'A20'})
  request = requests.get(url + queryParams)
  json_data = json.loads(request.text)

  if feature == 'getUVIdx':
    return json_data["response"]['body']["items"]["item"][0]['today']
  else:
    return json_data["response"]['body']["items"]["item"][0]['h3']

def get_living_weather_data(key, query, time, lasttime,result, areano='1100000000'):
  print("Get living weather data...")
  for feature in query:
    result[feature] = api_Living_Reading(key, feature, time)
  print("Done.")
  return result, lasttime

def api_air_pullution(key, item_code):
  # Used api made by Korea Environment Corporation
  # https://data.go.kr/tcs/dss/selectApiDataDetailView.do?publicDataPk=15000581
  url = f"http://openapi.airkorea.or.kr/openapi/services/rest/ArpltnInforInqireSvc/getCtprvnMesureLIst"
  #-*- coding:utf-8 -*-
  queryParams = '?' + urlencode({ quote_plus('ServiceKey') : key, quote_plus('itemCode') : item_code, quote_plus('dataGubun') : 'HOUR', quote_plus('_returnType') : 'json' })
  request = requests.get(url + queryParams)

  json_data = json.loads(request.text)
  return json_data['list'][0]

def get_air_pollution_data(key, item_code, result, station="seoul"):
  print("Get air pollution data...")
  for item in item_code:
    value = api_air_pullution(key, item)
    result[item] = value[station]
  print("Done.")

  return result

def get_all_api_data(date, lasttime):
  DECODE_KEY = unquote('cELHrfnETp70pjPK90A%2F%2B2h4u9%2FNYVQ6o4e%2B1icLTH6TodaaaxmUvLfsCsRVvH4XvI6NqZSl4jC3grykXKEVTg%3D%3D')
  DATE = date
  QUERY = ['getHeatFeelingIdx','getDiscomfortIdx','getUVIdx','getSenTaIdx','getAirDiffusionIdx']
  ITEM_CODE = ['SO2', 'CO', 'O3', 'NO2', 'PM10', 'PM25']
  result = {}

  result, lasttime = get_living_weather_data(DECODE_KEY, QUERY, DATE, lasttime, result)
  result = get_air_pollution_data(DECODE_KEY, ITEM_CODE, result, station='seoul')

  return result, lasttime


현재 시간을 받아와서 지금 시간을 갱신합니다.
코랩 서버가 미국 시간이라 timezone이 맞지 않습니다.

In [5]:
lasttime = "" # save the last operated time

In [2]:
now = time.localtime()
date = f"{now.tm_year}{now.tm_mon:02d}{now.tm_mday:02d}{now.tm_hour:02d}"

시간을 비교해 각 시간마다 한 번씩 API를 작동시킵니다.

In [3]:
if lasttime != date:
  print("Time passed.\nGet the api data")
  result, lasttime = get_all_api_data(date, lasttime)
  lasttime = date
  result['collected_time'] = lasttime
  print(result)
else:
  print("Print just past data")
  print(result)

Time passed.
Get the api data
Get living weather data...
Done.
Get air pollution data...
Done.
{'getHeatFeelingIdx': '24', 'getDiscomfortIdx': '74', 'getUVIdx': '', 'getSenTaIdx': '24', 'getAirDiffusionIdx': '100', 'SO2': '0.002', 'CO': '0.3', 'O3': '0.048', 'NO2': '0.016', 'PM10': '28', 'PM25': '19', 'collected_time': '2020073105'}


In [4]:
result

{'CO': '0.3',
 'NO2': '0.016',
 'O3': '0.048',
 'PM10': '28',
 'PM25': '19',
 'SO2': '0.002',
 'collected_time': '2020073105',
 'getAirDiffusionIdx': '100',
 'getDiscomfortIdx': '74',
 'getHeatFeelingIdx': '24',
 'getSenTaIdx': '24',
 'getUVIdx': ''}